In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
cd /content/drive/MyDrive/690F

[Errno 2] No such file or directory: '/content/drive/MyDrive/690F'
/Users/arpita/Downloads


In [1]:
import string
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("final_processed_train_data.csv")

In [3]:
train_df.shape

(1804475, 49)

In [4]:
train_df.columns
train_df.dropna(inplace = True)
to_show = ['id', 'target',
       'comment_text', 'asian', 'atheist', 'bisexual', 'black', 'buddhist',
       'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']
train_df.head().loc[:, to_show]

,id,target,comment_text,asian,atheist,bisexual,black,buddhist,christian,female,...,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white
11,239579,0.4400,This great story Man I wonder person yelled sh...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,239607,0.9125,Yet Muslims acts pilloried So okay smear entir...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,239644,0.0000,Because people drive cars ones cause wear tear...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,239653,0.3000,Mormons complicated relationship federal law,0.0,0.0,0.0,0.0,0.0,0.2,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0
111,239744,0.0000,I thing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# # train_df = train_df_whole.sample(n=5000, random_state=1)
# train_df = pd.read_csv("processed_train_df.csv")
# print(len(train_df))

In [6]:
train_df = train_df[train_df['target'] >= 0.20]

In [7]:
# the number of non toxic comments was more than 10 times the number of toxic comments, we have subsampled it to the following distribution
print("Number of non toxic comments: "+str(len(train_df[train_df["target"] < 0.5])))
print("Number of toxic comments: "+str(len(train_df[train_df["target"] >= 0.5])))

Number of non toxic comments: 41566
Number of toxic comments: 25142


In [8]:
train_df["prot_word_count"][train_df["prot_word_count"] > 0] = 1

In [9]:
print("Number of comments with bias inducing words not present: " + str(len(train_df[train_df["prot_word_count"] == 0])))
print("Number of comments with bias inducing words present: " + str(len(train_df[train_df["prot_word_count"] == 1])))

Number of comments with bias inducing words not present: 28700
Number of comments with bias inducing words present: 38008


In [10]:
X = train_df[["comment_text", "prot_word_count", "edited_comment_text"]]
y = np.array(train_df['target'].apply(lambda x: x > .5).astype('int'))

In [11]:
print(len(X))

66708


In [12]:
print(len(y))

66708


In [13]:
# Baseline models - Learning with Z

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
Xvec = vectorizer.fit_transform(X['comment_text'])
X_vect = pd.DataFrame(Xvec.todense(), columns=vectorizer.get_feature_names_out())

In [15]:
X_vect

,aa,aaa,aaaaaa,aaaaaaahthe,aaaaannnd,aaaaarrrrgggghhhh,aaaaerrrthat,aaaand,aaaannndddstill,aaah,...,𝙚𝙖𝙧𝙩𝙝,𝙣𝙤,𝙩𝙝𝙚,𝙩𝙝𝙚𝙧𝙚,𝙪𝙣𝙩𝙞𝙡,𝙬𝙖𝙡𝙠𝙚𝙙,𝙬𝙚𝙧𝙚,𝙮𝙚𝙖𝙧𝙨,𝟒𝟔,𝟔𝟎𝟎
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66703,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66704,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
prot_attr = ['transgendered', 'sex', 'bigotry', 'racist', 'muslim', 'racism', 'priest', 'black', 'sexual', 'latino', 'man', 'islam', 'gay', 'men', 'straight', 'transgender', 'homosexual', 'jewish', 'catholic', 'heterosexual', 'women', 'church', 'religious', 'military', 'asian', 'hindu', 'buddhist', 'religion', 'atheist', 'white', 'chinese', 'male', 'christian', 'race', 'bisexual', 'jew', 'woman']

In [ ]:
word_counts = {}
cols_to_drop = []
for i in list(X_vect.columns):
  X_vect[i][X_vect[i] > 0] = 1
  col_total = X_vect[i].sum()
  word_counts[i] = col_total
  if col_total < 11:
    cols_to_drop.append(i)

print("colstodrop",len(cols_to_drop))
print("orig_cols",len(X_vect.columns))
X_vect.drop(cols_to_drop, axis=1, inplace = True)
print("new_cols",len(X_vect.columns))

In [ ]:
print(len(X_vect))

In [ ]:
train_df['result'] = train_df['target'].apply(lambda x: x > .5).astype('int')

In [ ]:
train_df.to_csv("train_df.csv")

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=.4, stratify = y)

In [14]:
X_vect.to_csv("X_vect.csv")

In [16]:
from sklearn.linear_model import LogisticRegression
# Fitting a simple Logistic Regression on tf-idf
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
print("Accuracy: "+str(lr.score(X_test, y_test)))
y_preds = lr.predict(X_test)


Accuracy: 0.755


In [18]:
coeffs_ml_with_z = pd.DataFrame({"Feature" : X_vect.columns.tolist(),"Coefficients" : lr.coef_[0]})

In [21]:
prot_attr = ['transgendered', 'sex', 'bigotry', 'racist', 'muslim', 'racism', 'priest', 'black', 'sexual', 'latino', 'man', 'islam', 'gay', 'men', 'straight', 'transgender', 'homosexual', 'jewish', 'catholic', 'heterosexual', 'women', 'church', 'religious', 'military', 'asian', 'hindu', 'buddhist', 'religion', 'atheist', 'white', 'chinese', 'male', 'christian', 'race', 'bisexual', 'jew', 'woman']

In [23]:
coeffs_ml_with_z[coeffs_ml_with_z['Feature'].isin(prot_attr)]

,Feature,Coefficients
770,asian,-0.120542
848,atheist,0.542007
1255,bigotry,0.142672
1290,bisexual,-0.516810
1306,black,0.852846
1589,buddhist,1.585038
1839,catholic,-0.194096
2038,chinese,0.442887
2059,christian,-0.058949
2076,church,-0.037630


In [27]:
res = coeffs_ml_with_z[coeffs_ml_with_z['Feature'].isin(prot_attr)]
res.to_csv("coeffs_baseline_lr.csv")

In [26]:
groups = {
'religion' : ['religion', 'religious', 'atheist', 'buddhist', 'christian', 'hindu', 'jewish', 'muslim', 'catholic', 'church', 'islam', 'jew', 'priest'],
'race' : ['asian', 'black', 'latino', 'white', 'chinese', 'race', 'racism'],
'gender' : ['female', 'male', 'transgender', 'man', 'men', 'trangendered', 'woman', 'women'],
'sexual_identity' : ['bisexual', 'heterosexual', 'gay', 'homosexual', 'sex', 'sexual', 'straight' ],
'misc' : ['bigotry', 'military']
}

avg_coeffs = {}

for group in groups:
    avg_coeff = np.mean(np.array(coeffs_ml_with_z[coeffs_ml_with_z['Feature'].isin(groups[group])]['Coefficients']))
    avg_coeffs[group] = avg_coeff

avg_coeffs
    

{'religion': 0.20990736801478974,
 'race': 0.2827997845558042,
 'gender': -0.05647582212934848,
 'sexual_identity': 0.43078027628355364,
 'misc': 0.06597298218433253}